# Train the model

A preprocessed Nettiauto dataset is used to train the XGBoost model. The model and accompanying data is exported to separate files. The API server uses these files.

This notebook needs `nettiauto_dataset.csv` file created by `nettiauto_model.ipynb` notebook.

Running this notebook creates these files:

- `feature_names.json`
- `model.json`
- `example_X.json`
- `example_y.json`

In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option('display.max_columns', None)


### Hide warnings

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Load preprocessed dataset

The dataset file has been created in the: `nettiauto_dataset_parsing.ipynb`

In [3]:
df = pd.read_csv('./nettiauto_dataset.csv')
# df = df.drop('color', axis=1)

df.head(25)

,make,model,color,driveType,price,totalOwners,kilometers,seats,power,batteryCapacity,age,isSuv,metallicColor,airconditioning,parking_sensors,satellite_navigator,cruise_control_adaptive,parking_camera_simple_camera,seat_heaters,driving_assistant,alloy_wheels,electric_mirrors,lane_departure_warning_system,electrically_operated_tailgate,leather_upholstery,emergency_brake_assist,collision_avoidance_system,cruise_control_traditional,parking_assistant,tow_bar,electric_seats_with_memory,heated_steering_wheel,sunroof,sun_hatch_with_panorama,adaptive_headlights,sport_seats,fuel_battery_powered_heater,parking_camera_360-degree_camera,battery_preheating,electric_seats_without_memory,air_suspension,curve_lights,head_up_display,sport_base
0,porsche,taycan,black,rwd,101880.0,1,2100.0,5,300.0,79.00,2.416667,0,0,1,1,1,0,0,1,0,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
1,mercedes-benz,eqs,gray,4wd,125900.0,1,30000.0,5,385.0,108.40,2.000000,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,1,1,1,1,1,0,1,0,0,0
2,nissan,leaf,black,fwd,35800.0,1,10000.0,5,160.0,62.00,2.000000,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,citroen,e-c4,blue,fwd,33800.0,1,17000.0,5,100.0,50.00,2.000000,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,hyundai,kona,white,fwd,27590.0,1,56000.0,5,150.0,64.00,2.750000,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,1,0
5,citroen,e-c4,beige,fwd,29700.0,1,34000.0,5,100.0,50.00,2.000000,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,polestar,2,black,4wd,60780.0,1,7000.0,5,300.0,75.00,2.000000,0,0,1,1,1,1,0,1,1,1,1,0,1,0,1,1,0,1,0,1,1,1,0,1,1,1,1,0,0,0,1,0,0
7,peugeot,e-2008,orange,fwd,25790.0,1,13000.0,5,100.0,50.00,2.500000,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,bmw,ix3,white,rwd,51800.0,1,17000.0,5,210.0,80.00,2.166667,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
9,bmw,ix3,white,rwd,49900.0,1,25500.0,5,210.0,80.00,2.000000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0


## Boolean conversion

- Accessories 0/1 values (present/not-present), such as metallicColor = 1 or 0
- New columns are created for different *Make, model, color* and *driveType* strings and exclusive boolean values given for each

In [4]:
# Change Accessories 0/1 integer values to booleans
df.iloc[:, 11:] = df.iloc[:, 11:].astype(bool)

# Convert to make, model, color and dataType strings to distinct boolean values
df_encoded = pd.get_dummies(df, columns=[
    "make",
    "model",
    "color",
    "driveType"
])

df_encoded.head(25)


,price,totalOwners,kilometers,seats,power,batteryCapacity,age,isSuv,metallicColor,airconditioning,parking_sensors,satellite_navigator,cruise_control_adaptive,parking_camera_simple_camera,seat_heaters,driving_assistant,alloy_wheels,electric_mirrors,lane_departure_warning_system,electrically_operated_tailgate,leather_upholstery,emergency_brake_assist,collision_avoidance_system,cruise_control_traditional,parking_assistant,tow_bar,electric_seats_with_memory,heated_steering_wheel,sunroof,sun_hatch_with_panorama,adaptive_headlights,sport_seats,fuel_battery_powered_heater,parking_camera_360-degree_camera,battery_preheating,electric_seats_without_memory,air_suspension,curve_lights,head_up_display,sport_base,make_audi,make_bmw,make_byd,make_citroen,make_cupra,make_dacia,make_ds,make_fiat,make_ford,make_hyundai,make_jaguar,make_kia,make_lexus,make_maxus,make_mazda,make_mercedes-benz,make_mg,make_mini,make_muu merkki,make_nissan,make_opel,make_peugeot,make_polestar,make_porsche,make_renault,make_seat,make_skoda,make_subaru,make_tesla,make_toyota,make_volkswagen,make_volvo,model_2,model_208,model_3,model_500e,model_ariya,model_atto3,model_born,model_bz4x,model_c4,model_c40,model_citigo,model_combo-e,model_cooper,model_corsa-e,model_e-2008,model_e-208,model_e-308,model_e-berlingo,model_e-c4,model_e-c4 x,model_e-niro,model_e-soul,model_e-spacetourer,model_e-traveller,model_e-tron,model_e-tron gt,model_e-tron gt rs,model_e-up!,model_enyaq,model_eqa,model_eqb,model_eqc,model_eqe,model_eqs,model_eqs suv,model_eqv,model_euniq,model_ev6,model_ev9,model_han,model_i-pace,model_i3,model_i3s,model_i4,model_i4 m50,model_i7,model_id. buzz,model_id3,model_id4,model_id5,model_ioniq 5,model_ioniq 6,model_ioniq electric,model_ix,model_ix m60,model_ix1,model_ix3,model_kona,model_kona electric,model_leaf,model_marvel r,model_megane,model_mg4,model_mg5,model_mifa 9,model_mii electric,model_model 3,model_model s,model_model x,model_model y,model_mokka,model_mokka-e,model_mustang,model_mustang mach-e,model_muu malli,model_mx-30,model_niro,model_proace city verso ev,model_proace verso ev,model_q4 e-tron,model_q8 e-tron,model_solterra,model_spring,model_t90,model_tang,model_taycan,model_twingo,model_up!,model_ux,model_xc40,model_zafira-e,model_zoe,model_zs,color_beige,color_black,color_blue,color_brown,color_copper,color_gray,color_green,color_orange,color_other,color_red,color_silver,color_turquoise,color_violet,color_white,color_yellow,driveType_4wd,driveType_fwd,driveType_rwd
0,101880.0,1,2100.0,5,300.0,79.00,2.416667,False,False,True,True,True,False,False,True,False,True,True,False,True,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,125900.0,1,30000.0,5,385.0,108.40,2.000000,False,False,True,True,True,True,False,True,True,True,True,True,True,True,False,True,False,True,False,True,False,False,True,True,True,True,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

## Model training

### Test split

In [5]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop('price', axis=1)
y = df_encoded['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape

((3184, 182), (796, 182))

### XGBoost fit

In [6]:
import xgboost as xgb

xgb_regressor = xgb.XGBRegressor(objective ='reg:squarederror',
                                 n_estimators=100,
                                 max_depth=8,
                                 reg_lambda=1, # näitä voi säädellä muttei kauheesti oo vaikuttanut malliin
                                 reg_alpha=0,# näitä voi säädellä muttei kauheesti oo vaikuttanut malliin
                                 num_parallel_tree=5, # random forest 🤷‍♂️
)

xgb_regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=5, random_state=None, ...)

### Export the model and feature list

In [7]:
import json


### Export the model
xgb_regressor.save_model("model.json")


### Export feature names
# Feature names from the model
# column_names = list(xgb_regressor.feature_names_in_)

# Feature names from the test data frame
column_names = list(X.columns)

column_names = [ { "name": x } for x in column_names ]

with open("feature_names.json", "w") as f:
    json.dump(column_names, f)

### Export an example vector and a prediction

These can be used for testing or in a mock API

In [8]:
# Take the nth single row from X and y dataframes
n = 10

# Save a single row dataframe as json
example_X = X.iloc[[n]]
example_X.to_json("example_X.json", orient="split", index=False)

# Make a prediction and save as json
predicted_y = xgb_regressor.predict(example_X)
example_y = { "price": str(predicted_y[0]) }
with open("example_y.json","w") as f:
    json.dump(example_y, f)



## Test prediction

In [9]:
y_pred = xgb_regressor.predict(X_test)


### The mean absolute error

In [10]:
mean_absolute_error(y_test, y_pred)

3268.769992540829

### The mean squared error

In [11]:
mean_squared_error(y_test, y_pred)

28620479.38748582

### R² score

In [12]:
r2_score(y_test, y_pred)

0.9557884799582208

## Test predictions by make and model

In [13]:
def predict_make_model(make, model):
    m = (X_test[f'make_{make}'] == 1) & (X_test[f'model_{model}'] == 1)
    X_id3 = X_test[m]
    y_id3 = y_test[m]

    y_id3_pred = xgb_regressor.predict(X_id3)

    print('mean_absolute_error: {}'.format(mean_absolute_error(y_id3, y_id3_pred)))
    print('r2_score: {}'.format(r2_score(y_id3, y_id3_pred)))
    print()
    for real, pred in zip(y_id3, y_id3_pred):
        print(f'{real} -> {pred} ~ {real - pred} <> {pred / real}')

### Wolkswagen ID3

In [14]:
predict_make_model('volkswagen', 'id3')

mean_absolute_error: 1689.1068892045455
r2_score: 0.894975730164926

47000.0 -> 45248.91796875 ~ 1751.08203125 <> 0.9627429355053192
51355.0 -> 53238.91796875 ~ -1883.91796875 <> 1.0366842170918118
49390.0 -> 45544.77734375 ~ 3845.22265625 <> 0.9221457247165418
46700.0 -> 47821.671875 ~ -1121.671875 <> 1.024018669700214
45185.0 -> 46789.19140625 ~ -1604.19140625 <> 1.0355027421987386
30900.0 -> 31598.220703125 ~ -698.220703125 <> 1.0225961392597087
32490.0 -> 32174.2109375 ~ 315.7890625 <> 0.9902804228224069
33400.0 -> 31030.359375 ~ 2369.640625 <> 0.9290526758982036
42500.0 -> 44538.703125 ~ -2038.703125 <> 1.0479694852941177
43110.0 -> 44986.88671875 ~ -1876.88671875 <> 1.0435371542275573
49610.0 -> 45248.91796875 ~ 4361.08203125 <> 0.912092682296916
45815.0 -> 46431.22265625 ~ -616.22265625 <> 1.0134502380497654
44585.0 -> 46292.765625 ~ -1707.765625 <> 1.038303591454525
37900.0 -> 29445.83203125 ~ 8454.16796875 <> 0.776934882091029
44540.0 -> 44461.49609375 ~ 78.50390625 <> 0.99823

### BMW ix1

In [15]:
predict_make_model('bmw', 'ix1')

mean_absolute_error: 2841.0615234375
r2_score: 0.13013474953700999

59800.0 -> 66555.8359375 ~ -6755.8359375 <> 1.1129738451086957
62750.0 -> 70170.4765625 ~ -7420.4765625 <> 1.1182546065737051
69900.0 -> 71883.5859375 ~ -1983.5859375 <> 1.028377481223176
67890.0 -> 63887.28125 ~ 4002.71875 <> 0.9410411143025482
74310.0 -> 72986.3203125 ~ 1323.6796875 <> 0.9821870584376262
68720.0 -> 68836.0234375 ~ -116.0234375 <> 1.001688350371071
68540.0 -> 68836.0234375 ~ -296.0234375 <> 1.004318987999708
69800.0 -> 68969.8515625 ~ 830.1484375 <> 0.9881067559097422


### Tesla 3

In [16]:
predict_make_model('tesla', 'model 3')

mean_absolute_error: 2538.5584852430557
r2_score: 0.46078129000859036

35780.0 -> 35708.1171875 ~ 71.8828125 <> 0.9979909778507546
43950.0 -> 52649.62109375 ~ -8699.62109375 <> 1.1979435971274175
40500.0 -> 40954.6796875 ~ -454.6796875 <> 1.0112266589506174
39980.0 -> 37053.5078125 ~ 2926.4921875 <> 0.9268010958604302
43880.0 -> 43166.04296875 ~ 713.95703125 <> 0.9837293292787147
48700.0 -> 47831.8046875 ~ 868.1953125 <> 0.9821725808521561
39990.0 -> 35584.015625 ~ 4405.984375 <> 0.8898228463365842
39990.0 -> 40244.54296875 ~ -254.54296875 <> 1.0063651655101276
34990.0 -> 33209.66015625 ~ 1780.33984375 <> 0.9491186097813661
36800.0 -> 37376.53125 ~ -576.53125 <> 1.0156666100543479
39990.0 -> 41282.4921875 ~ -1292.4921875 <> 1.032320384783696
36900.0 -> 36695.83203125 ~ 204.16796875 <> 0.9944669927168022
42790.0 -> 43259.6171875 ~ -469.6171875 <> 1.0109749284295395
48490.0 -> 54406.9609375 ~ -5916.9609375 <> 1.1220243542482986
52990.0 -> 40463.05859375 ~ 12526.94140625 <> 0.763598010827

### Opel Mokka-e

In [17]:
predict_make_model('opel', 'mokka-e')

mean_absolute_error: 2550.29375
r2_score: 0.37129827343635435

32990.0 -> 36417.8046875 ~ -3427.8046875 <> 1.1039043554865111
37700.0 -> 38557.35546875 ~ -857.35546875 <> 1.0227415243700266
40180.0 -> 41095.5390625 ~ -915.5390625 <> 1.0227859398332504
42480.0 -> 39327.9609375 ~ 3152.0390625 <> 0.9257994570974576
44350.0 -> 42623.9765625 ~ 1726.0234375 <> 0.9610817714205186
42900.0 -> 36883.83984375 ~ 6016.16015625 <> 0.8597631665209791
36990.0 -> 39213.15625 ~ -2223.15625 <> 1.0601015477155988
38570.0 -> 39825.1484375 ~ -1255.1484375 <> 1.0325420906792844
44350.0 -> 42623.9765625 ~ 1726.0234375 <> 0.9610817714205186
38570.0 -> 39273.78125 ~ -703.78125 <> 1.0182468563650506
31990.0 -> 38354.7578125 ~ -6364.7578125 <> 1.198960856908409
44570.0 -> 42623.9765625 ~ 1946.0234375 <> 0.9563378183194974
44100.0 -> 40429.8359375 ~ 3670.1640625 <> 0.9167763251133787
38470.0 -> 39213.15625 ~ -743.15625 <> 1.019317812581232
41880.0 -> 38352.7265625 ~ 3527.2734375 <> 0.915776660995702


### Audi e-tron

In [18]:
predict_make_model('audi', 'e-tron')

mean_absolute_error: 7603.26171875
r2_score: 0.22519562957020722

64900.0 -> 82337.6015625 ~ -17437.6015625 <> 1.268684153505393
50990.0 -> 51653.37109375 ~ -663.37109375 <> 1.0130098272945676
50800.0 -> 58713.859375 ~ -7913.859375 <> 1.1557846333661417
66900.0 -> 63156.7890625 ~ 3743.2109375 <> 0.9440476690956652
67890.0 -> 71432.7109375 ~ -3542.7109375 <> 1.0521831041022243
54800.0 -> 64100.83203125 ~ -9300.83203125 <> 1.1697232122490875
79700.0 -> 92729.0859375 ~ -13029.0859375 <> 1.1634766115119197
43900.0 -> 48043.48828125 ~ -4143.48828125 <> 1.0943846988895216
50800.0 -> 49156.0859375 ~ 1643.9140625 <> 0.9676394869586614
48900.0 -> 63514.54296875 ~ -14614.54296875 <> 1.2988659093813906
